In [ ]:
import numpy as np
import random
import copy

In [ ]:
class solution():
  def __init__(self, n, p=0.5):
    self.vector = (p-0.1)*np.ones(n)
    self.taken = np.zeros(n)
    self.p=p
    self.prize = 0

  def decode(self):
    self.taken = self.vector > self.p
  
  def update(self, weights):
    self.prize = sum(self.taken*weights)


In [ ]:
class graph_():
  def __init__(self, edges, weights):
    self.n = len(weights)
    self.edges = copy.deepcopy(edges)
    self.weights = copy.deepcopy(weights)
    self.sumweights = sum(weights)
    self.deg = np.zeros(len(weights))
    self.adj = {i: [] for i in range(len(weights))}
    for edge in self.edges:
          self.deg[edge[0]]+=1
          self.adj[edge[0]].append(edge[1])
          if edge[0] != edge[1]:
            self.deg[edge[1]]+=1
            self.adj[edge[1]].append(edge[0])


In [ ]:
def is_feasible(elem, graph):
  for e in graph.edges:
    if not elem.taken[e[0]] and not elem.taken[e[1]]: return False
  return True

In [ ]:
class brkga():
  def __init__(self, graph, population_size=30, mutation_probability = .8, elitism = .1, generations = 30, p=0.5):
    self.graph = graph
    self.population_size = population_size
    self.mutation_probability = mutation_probability
    self.elitism = elitism
    self.generations = generations
    self.best_solution = solution(graph.n,p)
    self.p = p
    self.population = []
    self.best_gen = 0

  def initial_population(self):
    for i in range(self.population_size):
      new_element = solution(self.graph.n,0.5)
      edg = copy.deepcopy(graph.edges)
      np.random.shuffle(edg)
      for e in edg:
        if new_element.taken[e[0]] or new_element.taken[e[1]]: continue
        p0 = self.graph.weights[e[0]]/(self.graph.deg[e[0]]+.1)
        p1 = self.graph.weights[e[1]]/(self.graph.deg[e[1]]+.1)
        u01 = random.uniform(0,1)
        if u01 < p0/(p0+p1): 
          new_element.vector[e[1]]=self.p+0.1+np.random.normal(0,0.01)
          new_element.taken[e[1]] = 1
        else:
          new_element.vector[e[0]]=self.p+0.1+np.random.normal(0,0.01)
          new_element.taken[e[0]] = 1
      new_element.update(self.graph.weights)
      self.population.append(new_element)

  def parents_selection(self, t=5):
    parent1 = -1
    parent2 = -1
    prize1 = 0
    prize2 = 0
    for i in range(t):
      e = random.randint(0,self.population_size-1)
      if parent1==-1 or self.population[e].prize < prize1:
        prize1 = self.population[e].prize
        parent1 = e
    for i in range(t):
      e = random.randint(0,self.population_size-1)
      if parent2==-1 or self.population[e].prize < prize2:
        prize2 = self.population[e].prize
        parent2 = e
    return [self.population[parent1], self.population[parent2]]

  def mutation(self, elem):
    u01 = random.uniform(0,1)
    if u01<self.mutation_probability:
      for i in range(len(elem.vector)):
        if random.uniform(0,1) < .03:
          elem.vector[i] = random.uniform(0,1)
          if elem.vector[i] > self.p: elem.taken[i]=1
          else: elem.taken[i]=0
    else:
      MaxN = -1
      MinN = -1
      MaxE = -1
      MinE = -1
      for L in range(5):
        P = random.randint(0,len(elem.vector)-1)
        ratioP = self.graph.weights[P]/(self.graph.deg[P]+1)
        if MaxN == -1 or MaxN<ratioP:
          MaxN = ratioP
          MaxE = P
        if MinN == -1 or MinN>ratioP:
          MinN = ratioP
          MinE = P
      elem.vector[MaxE]=(1+elem.vector[MaxE])/2
      elem.vector[MinE]=elem.vector[MinE]/1.5
      elem.taken[MaxE] = elem.vector[MaxE] > self.p
      elem.taken[MinE] = elem.vector[MinE] > self.p

  def crossover(self):
    children = []
    for i in range(self.population_size//2):
      parents = self.parents_selection()
      child1 = solution(self.graph.n)
      child2 = solution(self.graph.n)
      for j in range(self.graph.n): 
        u01 = random.uniform(0,1)
        if u01 <0.5 : child1.vector[j] = (parents[0].vector[j] + parents[1].vector[j])/2;
        elif u01<0.75 : child1.vector[j] = parents[0].vector[j];
        else: child1.vector[j] = parents[1].vector[j];

      child1.decode()
      child1.update(self.graph.weights)
      self.mutation(child1)
      self.repairPut(child1)
      self.repairTake(child1)

      for j in range(self.graph.n):
        P1 = parents[0].prize
        P2 = parents[1].prize
        child2.vector[j] = (P1*parents[0].vector[j] + P2*parents[1].vector[j])/(P1+P2);
      child2.decode()
      child2.update(self.graph.weights)

      self.mutation(child2)
      self.repairPut(child2)
      self.repairTake(child2)

      children.append(child1)
      children.append(child2)
    
    return children
  
  def avoid_convergence(self):
    for L in range(int(self.population_size*0.1)):
      u = random.randint(0,self.population_size-1)
      v = random.randint(0,self.population_size-1)
      bb = True
      for i in range(self.graph.n):
        if self.population[u].taken[i]!=self.population[v].taken[i]: bb=False
      if u!=v and bb:
        self.mutation(self.population[u])
        self.mutation(self.population[u])
        self.mutation(self.population[u])
        self.repairPut(self.population[u])
        self.repairTake(self.population[u])
  
  def repairPut(self, elem):
      edg = []
      for e in self.graph.edges:
        if not elem.taken[e[0]] and not elem.taken[e[1]]: edg.append(e)
      edg = np.array(edg)
      np.random.shuffle(edg)

      for e in edg:
        if elem.taken[e[0]] or elem.taken[e[1]]:  continue
        p0 = self.graph.weights[e[0]]/(self.graph.deg[e[0]]+.1)
        p1 = self.graph.weights[e[1]]/(self.graph.deg[e[1]]+.1)
        u01 = random.uniform(0,1)
        if u01 < p0/(p0+p1): 
          elem.vector[e[1]]=random.uniform(self.p,1)
          elem.taken[e[1]]=1
        else: 
          elem.vector[e[0]]=random.uniform(self.p,1)
          elem.taken[e[0]]=1
          
      elem.update(self.graph.weights)

  def eliminate_redundant(self, redundant, elem):
      for u in self.graph.adj[elem]:
        if u in redundant: redundant.remove(u)
      if elem in redundant: redundant.remove(elem)

  def repairTake(self, elem):
      redundant = []
      cnt = np.zeros(len(elem.vector))

      for i in range(len(elem.vector)):
        if not elem.taken[i]: continue
        ct = 0
        for j in self.graph.adj[i]:
          if elem.taken[j]: ct+=1
          else: break
        if ct>0 and ct==self.graph.deg[i] and (i not in self.graph.adj[i]): redundant.append(i)

      while len(redundant):
        p = np.zeros(len(redundant))
        for i in range(len(redundant)):
          p[i] = self.graph.weights[i]/((self.graph.deg[i]+0.01)*self.graph.sumweights);
        p = np.exp(2*np.array(p))
        p = p/p.sum()

        E = np.random.choice(redundant, 1, p = p)[0]
        elem.taken[E] = 0
        elem.vector[E] = random.uniform(0,self.p)
        self.eliminate_redundant(redundant, E)

      elem.update(self.graph.weights)

  def next_generation_selection(self, parents, children):
      next_gen = []
      parents.sort(key = lambda x: x.prize)
      children.sort(key = lambda x: x.prize)

      for i in range(int(self.population_size*0.1)):
        next_gen.append(parents[i])
      for i in range(int(self.population_size*0.8)):
        next_gen.append(children[i])
      I = int(self.population_size*0.1)
      J = int(self.population_size*0.8)

      for i in range(int(self.population_size*0.1)):
        p0 = parents[I].prize
        p1 = children[J].prize
        u01 = random.uniform(0,1)
        if u01 < p0/(p1+p0):
          next_gen.append(children[J])
        else:
          next_gen.append(parents[I])
        I+=1
        J+=1
      return next_gen

  def solve(self):
    self.initial_population()
    self.best_solution = copy.deepcopy(self.population[0])

    counter = 0
    while (counter < self.generations):
      children = self.crossover() #Parents Selection, Crossover, Mutation, RepairPut e RepairTake
      self.population = self.next_generation_selection(copy.deepcopy(self.population), children)
      self.avoid_convergence()
      for individual in self.population:
        if individual.prize < self.best_solution.prize:
          self.best_solution = copy.deepcopy(individual)
          self.best_gen = counter
      counter+=1
      if (counter%20==0): print(counter,self.best_solution.prize)

  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Instances'

bio-celegans.mtx   gent113.mtx		 soc-douban.mtx
bio-diseasome.mtx  ia-email-EU.mtx	 soc-karate.mtx
bio-yeast.mtx	   ia-enron-only.mtx	 tech-as-caida2007.mtx
boyd2.mtx	   ia-reality.mtx	 web-edu.mtx
ca-CSphd.mtx	   lp1.mtx		 web-google.mtx
ca-Erdos992.mtx    road-chesapeake.mtx	 web-indochina-2004.mtx
can_96.mtx	   rt-retweet-crawl.mtx  web-polblogs.mtx
ca-netscience.mtx  rt-retweet.mtx	 web-webbase-2001.mtx
dwt_59.mtx	   rt-twitter-copen.mtx
GD95_c.mtx	   soc-dolphins.mtx


In [ ]:
from scipy.io import mmread
def toWE_mtx(name):
  end = f'/content/drive/MyDrive/Instances/{name}.mtx'
  a = mmread(end).toarray()
  w = [((i+2)%200) for i in range(len(a))]
  e = []
  for i in range(len(a)):
    for j in range(i,len(a)):
      if a[i][j]: e.append([i,j])
  return w,e

In [ ]:
names = ['ia-email-EU']

In [ ]:
import time

In [ ]:
for name in names:
  w,e = toWE_mtx(name)
  graph = graph_(e,w)
  GA = brkga(graph,generations=200)
  aux = time.time()
  GA.solve()
  print(f'{name} : {time.time()-aux}')

20 78470
40 78335
60 78310
80 78310
100 78310
120 78310
140 78310
160 78310
180 78310
200 78310
ia-email-EU : 2429.7700550556183
